In [21]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import imodelsx.process_results
import sys
import datasets
import numpy as np
from copy import deepcopy
import clin.eval.eval
import clin.eval.med_status
import clin.llm
import clin.parse
from collections import defaultdict
import openai
openai.api_key_path = '/home/chansingh/.OPENAI_KEY'

sys.path.append('../experiments/')
results_dir = '../results/'

r = imodelsx.process_results.get_results_df(results_dir, use_cached=False)
r = r[r.dataset_name == 'medication_status']

# get data for eval
dset = datasets.load_dataset('mitclinicalml/clinical-ie', 'medication_status')
df_val = pd.DataFrame.from_dict(dset['validation'])
df = pd.DataFrame.from_dict(dset['test'])
# df = pd.concat([val, test])
# nums = np.arange(len(df)).tolist()
# np.random.default_rng(seed=13).shuffle(nums)
# dfe = df.iloc[nums]
n = len(df)

r = clin.eval.med_status.add_status_eval(r, df)

[autoreload of pandas.core.arrays.timedeltas failed: Traceback (most recent call last):
  File "/home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "/home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 471, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/pandas/core/arrays/timedeltas.py", line 34, in <module>
    from pandas._libs.tslibs.fields import (
ImportError: cannot import name 'get_timedelta_days' from 'pandas._libs.t

In [19]:
r[(r.n_shots == 5) * (r.checkpoint == 'text-davinci-003')].iloc[:1]

,dataset_name,seed,save_dir,checkpoint,checkpoint_verify,n_shots,use_cache,save_dir_unique,list_original,list_ov,...,status_acc_cond___pv,status_f1_macro_cond___pv,status_acc_cond___ev,status_f1_macro_cond___ev,status_acc_cond___ov_pv,status_f1_macro_cond___ov_pv,status_acc_cond___ov_pv_ev,status_f1_macro_cond___ov_pv_ev,status_acc_cond___sv,status_f1_macro_cond___sv
1,medication_status,1,/home/chansingh/clinical-extract-and-check/res...,text-davinci-003,text-davinci-003,5,1,/home/chansingh/clinical-extract-and-check/res...,NaN,NaN,...,0.894569,0.665284,0.894569,0.665284,0.897764,0.667529,0.897764,0.667529,0.907348,0.770679


In [20]:
# compare values for a single row
row_df = r[(r.n_shots == 5) * (r.checkpoint == 'text-davinci-003')].iloc[:1].reset_index()
rc = row_df[[c for c in row_df.columns if '___' in c]]
# create multindex columns by splitting on '___'
rc = rc.rename(columns={c: tuple(c.split('___')) for c in rc.columns})

# convert tuple column names to multiindex
rc.columns = pd.MultiIndex.from_tuples(rc.columns)
rc = rc.T.reset_index()
rc = rc.rename(columns={
    'level_0': '',
    'level_1': 'Verifiers',
}).pivot_table(index='Verifiers', columns='', values=0).round(3)
rc.style.format(precision=3).background_gradient(cmap='gray')

,f1,precision,recall,status_acc_cond,status_f1_macro_cond
Verifiers,,,,,
ev,0.921,0.921,0.921,0.895,0.665
original,0.922,0.921,0.924,0.895,0.665
ov,0.906,0.868,0.947,0.898,0.668
ov_pv,0.939,0.946,0.932,0.898,0.668
ov_pv_ev,0.939,0.949,0.929,0.898,0.668
pv,0.930,0.948,0.912,0.895,0.665
sv,0.939,0.949,0.929,0.907,0.771


In [17]:
cols = {
    'f1': 'F1',
    'precision': 'Precision',
    'recall': 'Recall',
    'status_f1_macro_cond': 'F1 (Medication Status)',
}
rows = {
    'original': 'Original',
    'ov': 'Omission',
    'pv': 'Prune',
    'ov_pv': 'Omission + Prune',
    'sv': 'Omission + Prune + Evidence'
}
(
    rc[list(cols.keys())].rename(columns=cols)
    .loc[list(rows.keys())].rename(index=rows)
    .style.format(precision=3).background_gradient(cmap='Blues')
)


,F1,Precision,Recall,F1 (Medication Status)
Verifiers,,,,
Original,0.922,0.921,0.924,0.665
Omission,0.906,0.868,0.947,0.668
Prune,0.930,0.948,0.912,0.665
Omission + Prune,0.939,0.946,0.932,0.668
Omission + Prune + Evidence,0.939,0.949,0.929,0.771


In [ ]:
# print errs for med extraction
# mets = clin.eval.calculate_metrics(row['dict_ov_pv_ev'], df, verbose=True)

# print errors for status extraction
# accs_cond, f1s_macro_cond = clin.eval.eval_medication_status([med_status_dicts_list[-1]], df, verbose=True)

## Interpret an example

In [59]:
# single row to investigate
row = r[(r.n_shots == 5) * (r.checkpoint == 'text-davinci-003')].iloc[0]

In [81]:
i = 11
ev = row['dict_evidence_ov_pv_ev'][i]
med_status_dict = row['dict_sv'][i]
gt = df.iloc[i]
print(gt)
print(med_status_dict)
print('\n' + gt['snippet'])

for k in ev:
    print(k)
    print('\t', ev[k])
    print()

index                                                                      18
snippet                     Her aspirin (81 mg q.d.) is discontinued, and ...
active_medications                        ["Plavix", "Tylenol", "Tylenol ES"]
discontinued_medications                                          ["aspirin"]
neither_medications                                 ["ibuprofen", "Naprosyn"]
Name: 13, dtype: object
{'aspirin': 'discontinued', 'ibuprofen': 'neither', 'Naprosyn': 'neither', 'Tylenol': 'active', 'Plavix': 'active'}

Her aspirin (81 mg q.d.) is discontinued, and the patient is advised that she needs to avoid ibuprofen, Naprosyn, alcohol, caffeine, and chocolate. She is advised that Tylenol 325 mg or Tylenol ES (500 mg) is safe to take at 1 or 2 q.4-6h. p.r.n. for pain or fever. Discharge activity is without restriction. DISCHARGE MEDICATIONS: 1. Plavix 75 mg p.o. q.d.
aspirin
	 her aspirin (81 mg q.d.) is discontinued

ibuprofen
	 avoid ibuprofen

Naprosyn
	 avoid naprosyn

Tyl